In [1]:
import pandas as pd
import geopandas as gpd
import shapely.geometry as geom
import numpy as np
import json

In [2]:

gdf : gpd.GeoDataFrame = gpd.read_file('local/MB_2021_AUST_SHP_GDA2020/MB_2021_AUST_GDA2020.shp')
# 4m

In [3]:
gdf : gpd.GeoDataFrame = gdf[['MB_CODE21', 'geometry']].rename(columns={'MB_CODE21': 'id'})

In [4]:
gdf_geom : gpd.GeoSeries = gdf.boundary
# 30s

In [5]:
# Combine multiple linestring and multilinestring geometries into single multilinestring geometry
gdml = gdf_geom.apply(lambda x: geom.MultiLineString([x]) if isinstance(x, geom.LineString) else x)
# 1m

In [6]:
gdl = gdml.apply(lambda x: x.geoms if x is not None else None).explode()
# 1m

In [7]:
gdl.dropna(inplace=True)

In [8]:
coords = gdl.apply(lambda x: np.array(x.coords))
# 1m

In [9]:
# Get all points
points_full = np.concatenate(coords.to_numpy())
# points_full.sort(axis=0)
len(points_full)
# 19647448
# 10s

19647448

In [10]:
# # Drop duplicates
# points = np.unique(points_full, axis=0)
# len(points)
# # 14983534

# # Split long and lat into separate columns, and save to csv, with index and header
# df = pd.DataFrame(points, columns=['long', 'lat'])
# df.to_csv('local/mb_points.csv', index=True, header=True)
# # 1m 30s

# pd.read_csv('local/mb_points.csv', index_col=0)
# # 10s

In [11]:
df_full = pd.DataFrame(points_full, columns=['long', 'lat'])
dfx = df_full.drop_duplicates().reset_index(drop=True)
# 20s

In [ ]:
# coords.apply(lambda line: [dfx[(dfx['long'] == x[0]) & (dfx['lat'] == x[1])].index[0] for x in line])
# # 0.5s per line -> 300,000 lines -> 48 hours

In [ ]:
[dfx[(dfx['long'] == x[0]) & (dfx['lat'] == x[1])].index[0] for x in coords[0]]

In [ ]:
coords[:5].apply(lambda line: [dfx[(dfx['long'] == x[0]) & (dfx['lat'] == x[1])].index[0] for x in line])

In [ ]:
dfx.set_index(['long', 'lat'], inplace=False, drop=False)
# 1m